In [5]:
import pandas as pd
import pathlib as pl
file = pl.Path("../recieved.csv")

headerNames = ["time", "recieverId", "tempInC", "pressure", "altitude", "humidity", "capacitance"]
recievedDf = pd.read_csv(file, names=headerNames)  # dF means DataFrame, a construct in pandas for tabled data
print(recievedDf)

                          time  recieverId  tempInC  pressure  altitude  \
0   2022-01-24 12:03:11.113422           1    24.15    988.38    209.11   
1   2022-01-24 12:03:22.126654           1    24.13    988.40    208.96   
2   2022-01-24 12:03:33.139828           1    24.13    988.42    208.83   
3   2022-01-24 12:03:44.142931           1    24.13    988.39    209.06   
4   2022-01-24 12:03:53.132063           1    24.14    988.39    209.05   
5   2022-01-24 12:04:04.151023           1    24.12    988.43    208.74   
6   2022-01-24 12:04:15.167402           1    24.13    988.36    209.33   
7   2022-01-24 12:04:24.153225           1    24.15    988.39    209.03   
8   2022-01-24 12:04:35.171424           1    24.15    988.37    209.19   
9   2022-01-24 12:04:46.196605           1    24.16    988.40    209.01   
10  2022-01-24 12:04:57.204742           1    24.15    988.38    209.12   
11  2022-01-24 12:05:06.202570           1    24.15    988.38    209.19   

    humidity  capacitanc

In [6]:
import plotly.express as px

px.scatter(recievedDf, x="time", y="capacitance")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'time=%{x}<br>capacitance=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['2022-01-24 12:03:11.113422', '2022-01-24 12:03:22.126654',
                          '2022-01-24 12:03:33.139828', '2022-01-24 12:03:44.142931',
                          '2022-01-24 12:03:53.132063', '2022-01-24 12:04:04.151023',
                          '2022-01-24 12:04:15.167402', '2022-01-24 12:04:24.153225',
                          '2022-01-24 12:04:35.171424', '2022-01-24 12:04:46.196605',
                          '2022-01-24 12:04:57.204742', '2022-01-24 12:05:06.202570'],
                         dtype=object),
              'xaxis': 'x',
              'y': array([377, 377, 377, 377, 377, 377, 377, 377, 377, 377, 377, 377]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'time'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'capacitance'}}}
})